In [1]:
import pandas as pd
import json
import configparser
from pathlib import Path
import os

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SimpleRNN

configs = configparser.ConfigParser()
configs.read("../../configuracoes.conf",encoding='utf-8')
data_folder = Path(configs["DEFAULT"]["folder_path"])
path = data_folder / configs["DEFAULT"]["file"]
objeto = configs["DEFAULT"]["objeto"]
regiao = configs["DEFAULT"]["regiao"]

look_back = 12 #Quantidade de meses anteriores usado para prever
forecast_horizon = int(configs["DEFAULT"]["meses"]) #Quantidade de meses a serem previstos

# Prever Cesta

In [2]:
#Carrega os dados
cesta = pd.read_excel(path)
cesta.drop(['ano'], axis=1, inplace=True)
cesta = cesta / 1000
cesta = cesta.iloc[::, ::]

#Cria sequencias de tempo de tamanho (meses anteriores + meses a serem previstos)
for n_step in range(1, look_back + forecast_horizon):
    cesta[f'preco t(h + {n_step})'] = cesta['preco'].shift(-n_step).values
cesta.dropna(inplace=True)
cesta.reset_index(drop=True, inplace=True)

#Divide os dados em Treino e Teste
X_train = cesta.iloc[::, :look_back:].values
y_train = cesta.iloc[::, look_back::].values
X_val = cesta.iloc[-1::, -12::].values

#remodela os dados para formato (batches, sequencia de tempo, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

#Cria o modelo
model=Sequential()
model.add(SimpleRNN(24, stateful=True, input_shape=(look_back, 1), batch_input_shape=(1, look_back, 1)))
model.add(Dense(forecast_horizon))
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))

#Treina e prever (forecast_horizon) quantidade de tempo
history = model.fit(X_train, y_train, epochs= 150, batch_size=1, shuffle=False)
forecast = model.predict(X_val, batch_size=1)

results = [value for value in forecast[0]]
objeto = objeto.replace(" ",'_')
output = {objeto.lower():str(results)}

if forecast_horizon == 12:
    file_name = f"previsao_RNN_12_meses_{objeto.lower()}_{regiao.lower()}.json"
else:
    file_name = f"previsao_RNN_{objeto.lower()}_{regiao.lower()}.json"
    
full_url = f"../../output/previsoes_cesta/{file_name}"
os.makedirs(os.path.dirname(full_url), exist_ok=True)
with open(full_url,"w") as file:
    json.dump(output,file,ensure_ascii=False)
print("Arquivo com resultados gerados")

Epoch 1/150
205/205 [==============================] - 0s 2ms/step - loss: 0.0062
Epoch 2/150
205/205 [==============================] - 0s 1ms/step - loss: 0.0035
Epoch 3/150
205/205 [==============================] - 0s 1ms/step - loss: 0.0019
Epoch 4/150
205/205 [==============================] - 0s 1ms/step - loss: 0.0013
Epoch 5/150
205/205 [==============================] - 0s 1ms/step - loss: 0.0012
Epoch 6/150
205/205 [==============================] - 0s 1ms/step - loss: 0.0016
Epoch 7/150
205/205 [==============================] - 0s 1ms/step - loss: 0.0012
Epoch 8/150
205/205 [==============================] - 0s 1ms/step - loss: 0.0012
Epoch 9/150
205/205 [==============================] - 0s 1ms/step - loss: 0.0012
Epoch 10/150
205/205 [==============================] - 0s 1ms/step - loss: 0.0011
Epoch 11/150
205/205 [==============================] - 0s 1ms/step - loss: 0.0010
Epoch 12/150
205/205 [==============================] - 0s 1ms/step - loss: 9.1919e-04
Epoch 13/

# Prever Produtos

In [2]:
path_inicial = "../../data/datasets_produtos/"

regioes = ['ilheus','itabuna']
produtos = ['acucar', 'arroz', 'banana', 'cafe', 'carne', 'farinha', 'feijao', 'leite', 
            'manteiga', 'oleo', 'pao', 'tomate']

for regiao in regioes:
    for objeto in produtos:
        #Carrega os dados
        path = f'{path_inicial}{regiao}/{objeto}_{regiao}.xlsx'
        cesta = pd.read_excel(path)
        cesta.drop(['ano'], axis=1, inplace=True)
        cesta = cesta / 1000
        cesta = cesta.iloc[::]

        #Cria sequencias de tempo de tamanho (meses anteriores + meses a serem previstos)
        for n_step in range(1, look_back + forecast_horizon):
            cesta[f'preco t(h + {n_step})'] = cesta['preco'].shift(-n_step).values
        cesta.dropna(inplace=True)
        cesta.reset_index(drop=True, inplace=True)

        #Divide os dados em Treino e Teste
        X_train = cesta.iloc[::, :look_back:].values
        y_train = cesta.iloc[::, look_back::].values
        X_val = cesta.iloc[-1::, -12::].values

        #remodela os dados para formato (batches, sequencia de tempo, features)
        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

        #Cria o modelo
        model=Sequential()
        model.add(SimpleRNN(24, stateful=True, input_shape=(look_back, 1), batch_input_shape=(1, look_back, 1)))
        model.add(Dense(forecast_horizon))
        model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))

        #Treina e prever (forecast_horizon) quantidade de tempo
        history = model.fit(X_train, y_train, epochs= 150, batch_size=1, shuffle=False)
        forecast = model.predict(X_val, batch_size=1)

        results = [value for value in forecast[0]]
        objeto = objeto.replace(" ", '_')
        output = {objeto.lower():str(results)}
        
        if forecast_horizon == 12:
            file_name = f"previsao_RNN_12_meses_{objeto.lower()}_{regiao.lower()}.json"
        else:
            file_name = f"previsao_RNN_{objeto.lower()}_{regiao.lower()}.json"
        
        full_url = f"../../output/previsoes_produtos/{regiao}/{file_name}"
        os.makedirs(os.path.dirname(full_url), exist_ok=True)
        with open(full_url,"w") as file:
            json.dump(output,file,ensure_ascii=False)
        print("Arquivo com resultados gerados")

Epoch 1/150
205/205 [==============================] - 0s 1ms/step - loss: 1.4127e-05
Epoch 2/150
205/205 [==============================] - 0s 1ms/step - loss: 6.4874e-06
Epoch 3/150
205/205 [==============================] - 0s 2ms/step - loss: 4.5573e-06
Epoch 4/150
205/205 [==============================] - 0s 1ms/step - loss: 4.3417e-06
Epoch 5/150
205/205 [==============================] - 0s 1ms/step - loss: 4.4255e-06
Epoch 6/150
205/205 [==============================] - 0s 1ms/step - loss: 4.6045e-06
Epoch 7/150
205/205 [==============================] - 0s 1ms/step - loss: 4.8445e-06
Epoch 8/150
205/205 [==============================] - 0s 1ms/step - loss: 4.5051e-06
Epoch 9/150
205/205 [==============================] - 0s 1ms/step - loss: 4.5904e-06
Epoch 10/150
205/205 [==============================] - 0s 1ms/step - loss: 4.8456e-06
Epoch 11/150
205/205 [==============================] - 0s 1ms/step - loss: 4.9184e-06
Epoch 12/150
205/205 [==============================